In [1]:
import langchain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.agents import load_tools, initialize_agent
import openai
import os
import wandb
from wandb.integration.langchain import WandbTracer
import json

In [2]:
wandb.init(
    project="generate_simple_prd",
    config={
        "model": "gpt-3.5-turbo",
        "temperature": 0
    },
    entity="arihantsheth",
    name="generate_multiple_prd_1"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arihantsheth. Use `wandb login --relogin` to force relogin


In [2]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

prompt_template = load_prompt("./prompt_templates/generate_prd_template.json")

c:\Users\ariha\Desktop\Synap Labs\synap_labs_env\lib\site-packages\langchain\llms\openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\ariha\Desktop\Synap Labs\synap_labs_env\lib\site-packages\langchain\llms\openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
prompt_input_variables = {
    "wandb": dict(
        new_feature="Model Explainability Toolkit (MXT)",
        new_feature_desc="Enables users to interpret and explain the outputs and decision-making of their AI Models."
    ),
    "langchain": dict(
        new_feature="Model Training Wizard (MTW)",
        new_feature_desc=" A user-friendly interface that guides users through the process of training their own custom language models. It would provide step-by-step instructions, dataset management tools, and performance evaluation metrics to help users create high-quality models tailored to their specific needs."
    )
}

In [5]:
string_template = """\
You are a talkative product manager.

Now, you want to add the following new feature to this:
{new_feature}
Feature description: {new_feature_desc}

Create a Product Requirement Document (PRD) to add this new feature in the following format:

1. Project Overview: An introduction to the project, its purpose, and the problem it aims to solve.

2. Project Scope: A detailed description of what is included and excluded from the project.

3. Functional Requirements: Specific features and functionalities that the project should deliver.

4. Non-functional Requirements: Criteria related to performance, security, usability, scalability, etc.

5. Use Cases: Detailed scenarios that describe how users will interact with the product.

6. User Interface (UI) and User Experience (UX) Guidelines: Design principles and requirements for the user interface.

7. Assumptions and Constraints: Factors or limitations that may impact the project's development or implementation.

8. Technical Requirements: Any specific technologies, platforms, or infrastructure required for the project.

9. Timeline and Milestones: A proposed schedule or timeline for the project, including key milestones and deliverables.

10. Success Criteria: Criteria for evaluating the success of the project and how it aligns with the overall business goals.

11. Customer analysis: An analysis of customers who would actually use this feature. 

Return your output in a markdown format to maintain consistency."""

In [12]:
# prompt_template = PromptTemplate(template=string_template, input_variables=["new_feature", "new_feature_desc"])

prompt_template.save("../prompt_templates/generate_prd_template_v1.json")

In [7]:
prompts = dict()
for company in prompt_input_variables:
    prompts[company] = prompt_template.format(**prompt_input_variables[company])

In [13]:
with open("../company_prompts/prompt_inputs.json", "w") as f:
    json.dump(prompt_input_variables, f)

with open("../company_prompts/prompts.json", "w") as f:
    json.dump(prompts, f)

In [8]:
outputs = dict()

for company in prompts:
    print(f"Generating PRD for {company}...")
    prompt = prompts[company]
    
    try:
        # output = llm(prompt, callbacks=[WandbTracer()])
        output = llm(prompt)
    except Exception as e:
        print(e)
        output = f"ERROR: {e}"

    outputs[company] = output
    # wandb.log({f"{company}_prd": output})

Generating PRD for wandb...
Generating PRD for langchain...


In [8]:
for company in outputs:
    with open(f"./generated_prds/{company}_prd.md", "w") as f:
        f.write(outputs[company])

In [10]:
wandb.finish()

langchain_prd,# Product Requiremen...
wandb_prd,# Product Requiremen...


In [10]:
from IPython.display import Markdown, display

display(Markdown(outputs["wandb"]))

# Product Requirement Document (PRD)

## 1. Project Overview:
The Model Explainability Toolkit (MXT) is a project aimed at enabling users to interpret and explain the outputs and decision-making of their AI Models. The purpose of this project is to provide users with a tool that enhances their understanding of AI models, allowing them to gain insights into the reasoning behind the model's predictions. By addressing the lack of transparency in AI models, MXT aims to increase trust and confidence in AI systems.

## 2. Project Scope:
The project includes the development of a toolkit that integrates with existing AI models and provides users with interpretability and explanation capabilities. The toolkit will support various types of AI models and will be compatible with popular frameworks and platforms. However, the project does not involve the development of new AI models or modifications to the existing models.

## 3. Functional Requirements:
- Integration: The toolkit should be able to integrate with different AI models and frameworks.
- Interpretability Metrics: Provide users with interpretability metrics to quantify the transparency of AI models.
- Feature Importance: Enable users to identify and understand the importance of different features in the model's decision-making process.
- Rule Extraction: Allow users to extract rules or decision trees from the AI model to gain insights into its decision-making process.
- Visualization: Provide visualizations to help users understand and interpret the outputs of the AI model.
- Explanation Generation: Generate explanations for individual predictions made by the AI model, highlighting the key factors that influenced the decision.

## 4. Non-functional Requirements:
- Performance: The toolkit should have minimal impact on the performance of the AI models.
- Security: Ensure the security and privacy of the AI models and their data.
- Usability: The toolkit should be user-friendly and intuitive, requiring minimal technical expertise.
- Scalability: The toolkit should be able to handle large-scale AI models and datasets efficiently.

## 5. Use Cases:
- Use Case 1: A data scientist wants to understand why an AI model classified a particular image as a cat and wants to visualize the important features that contributed to the decision.
- Use Case 2: A compliance officer needs to explain the decision-making process of an AI model to meet regulatory requirements and ensure transparency.
- Use Case 3: An AI model developer wants to compare the interpretability of different models to choose the most transparent one for a specific use case.

## 6. User Interface (UI) and User Experience (UX) Guidelines:
- The UI should be intuitive and provide clear navigation to different interpretability features.
- Visualizations should be interactive and allow users to explore different aspects of the model's decision-making process.
- Explanations should be presented in a concise and understandable manner, avoiding technical jargon.

## 7. Assumptions and Constraints:
- The toolkit assumes that users have access to trained AI models and their corresponding datasets.
- The interpretability provided by the toolkit is based on the characteristics and limitations of the underlying AI models.

## 8. Technical Requirements:
- The toolkit should be compatible with popular AI frameworks such as TensorFlow, PyTorch, and scikit-learn.
- It should support both on-premises and cloud-based deployment options.
- The toolkit should be able to handle different types of AI models, including deep learning models, decision trees, and ensemble models.

## 9. Timeline and Milestones:
- Milestone 1: Requirements gathering and analysis - 2 weeks
- Milestone 2: Development of interpretability metrics and feature importance - 4 weeks
- Milestone 3: Rule extraction and visualization - 6 weeks
- Milestone 4: Explanation generation and UI development - 4 weeks
- Milestone 5: Testing and bug fixing - 2 weeks

## 10. Success Criteria:
- The toolkit should provide users with meaningful and accurate interpretability insights for their AI models.
- Users should be able to easily integrate the toolkit into their existing AI workflows.
- The toolkit should receive positive feedback from users, indicating improved understanding and trust in AI models.

## 11. Customer Analysis:
The primary customers for this feature are data scientists, AI model developers, compliance officers, and stakeholders who require interpretability and explanation capabilities for their AI models. These customers are typically working in industries such as finance, healthcare, and legal, where transparency and accountability are crucial. The feature will be particularly valuable for organizations that need to comply with regulatory requirements or explain AI model decisions to end-users or auditors.